In [ ]:
from glob import glob
import os

import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

import threading

from src.utils.normalize_text import DocumentFormatter
from src.anexo_b.process_b_attachment import anexo_b

In [50]:
dir_Arquivos = Path(r"data\Input\Resumos")
output_pdf = Path(r"data\Output\Resumos\pdf")
output_excel = Path(r"data\Output\Resumos\excel")

In [3]:
itens = pd.read_csv(
    Path.joinpath(dir_Arquivos, "100 Kits_KPMG.csv"), dtype=str, names=["Contrato"]
)
comparativo = pd.read_excel(
    Path.joinpath(dir_Arquivos, "Status_2014.xlsx"), dtype={"Contrato": str}
)
caminho_base = Path.joinpath(dir_Arquivos, "modelo_piscofins.xlsx")
path_base = Path.joinpath(dir_Arquivos, "DOC_BASE_Reprocessado.xlsx")

In [4]:
df_quadro_um = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_um.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_dois = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_dois.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_tres = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_tres.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_quatro = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_quatro.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_cinco = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_cinco.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_seis = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_seis.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)

In [5]:
lst_itens = itens["Contrato"].tolist()

comparativo["Contrato"] = comparativo["Contrato"].astype(str)
comparativo = comparativo[comparativo["Contrato"].isin(lst_itens)]

## Processamento

In [6]:
tipos_colunas = {
    "nº do contrato": str,
    "Razão Social_Nome Completo do Arrendatário": str,
    "CNPJ_CPF do Arrendatário": str,
    "Nº de Parcelas Contratadas": str,
    "Valor do Bem": float,
}
data_columns = ["data do contrato", "Liquidação"]

df_base = pd.read_excel(path_base, dtype=tipos_colunas)
df_base["N° de parcelas contratadas"] = (
    pd.to_numeric(df_base["N° de parcelas contratadas"], errors="coerce")
    .fillna(0)
    .astype(int)
)

df_base["nº do contrato"] = df_base["nº do contrato"].str.zfill(7)

df_base["CNPJ_CPF do Arrendatário"] = df_base["CNPJ_CPF do Arrendatário"].apply(
    DocumentFormatter.format_documents
)

df_base["Razão Social_Nome Completo do Arrendatário"] = df_base[
    "Razão Social_Nome Completo do Arrendatário"
].apply(DocumentFormatter.to_pascal_case)
df_base = DocumentFormatter.format_date_columns(df_base, data_columns)
df_base["nº do contrato"] = df_base["nº do contrato"].astype(str)

In [7]:
df_base = df_base[
    [
        "nº do contrato",
        "Razão Social_Nome Completo do Arrendatário",
        "CNPJ_CPF do Arrendatário",
        "Valor do Bem",
        "data do contrato",
        "Liquidação",
        "N° de parcelas contratadas",
    ]
]
df_base.columns = [
    "Nº do Contrato",
    "Cliente",
    "CNPJ/CPF",
    "Valor do Bem",
    "Data do Contrato",
    "Data de Liquidação",
    "Quantidade de Parcelas",
]

In [8]:
contratos_validos = comparativo["Contrato"].values.tolist()

In [9]:
df_base["Nº do Contrato"].unique()

array(['0141715', '0158876', '0337640', ..., '5221444', '5221523',
       '5221862'], shape=(458118,), dtype=object)

In [10]:
df_base = df_base[df_base["Nº do Contrato"].isin(contratos_validos)]
info_contratos = df_base.groupby("Nº do Contrato")
replace = True

In [11]:
# warnings.filterwarnings("ignore")
thread_limit = 1
idx = 0
threads = []

pbar = tqdm(info_contratos, total=len(info_contratos))
for contrato, info in pbar:
    pbar.set_description(f"Processando contrato: {contrato}")
    thread = threading.Thread(
        target=anexo_b().criar_folha_rosto,
        args=(
            contrato,
            info,
            comparativo,
            output_excel,
            df_quadro_um,
            df_quadro_dois,
            df_quadro_tres,
            df_quadro_quatro,
            df_quadro_cinco,
            df_quadro_seis,
            caminho_base,
            replace,
        ),
    )
    thread.start()
    threads.append(thread)

    if len(threads) >= thread_limit or idx == len(info_contratos) - 1:
        # Aguarde até que todas as threads terminem
        for thread in threads:
            thread.join()
        threads = []
    idx += 1

  0%|          | 0/100 [00:00<?, ?it/s]

Exception in thread Thread-4 (criar_folha_rosto):
Traceback (most recent call last):
  File "C:\Users\jaquelinerufino\.pyenv\pyenv-win\versions\3.12.0\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\_projects\dev\superveniencia_piscofins\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\jaquelinerufino\.pyenv\pyenv-win\versions\3.12.0\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\_projects\dev\superveniencia_piscofins\src\anexo_b\process_b_attachment.py", line 1067, in criar_folha_rosto
    self._consolidate_workbooks_to_xlsx(
  File "c:\_projects\dev\superveniencia_piscofins\src\anexo_b\process_b_attachment.py", line 477, in _consolidate_workbooks_to_xlsx
    base_workbook.save(final_output_path)
  File "c:\_projects\dev\superveniencia_piscofins\.venv\Lib\site-packages\openpyxl\workbook\workbook.py", line 386, in save
    save_workbook(self

In [ ]:

files = glob(os.path.join(output_excel, "*.xlsx"))
output_pdf = os.path.abspath(output_pdf)

pbar = tqdm(files, total=len(files))
for file in pbar:
    pbar.set_description(f"Convertendo {os.path.basename(file)} para PDF")
    anexo_b().xlsx_to_pdf_one_page(os.path.abspath(file), output_pdf)

  0%|          | 0/100 [00:00<?, ?it/s]

PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0568553.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0568753.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0569107.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0569271.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0569332.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0569339.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0569340.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0569494.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0569646.pdf
PDF gerado em: C:\_projects\dev\superveniencia_piscofins\data\Output\resumos\pdf\RESUMO_0569809.pdf


In [102]:
anexo_b().process_anexo_pdfs(Path(output_pdf))

Processando grupos:   0%|          | 0/6 [00:00<?, ?it/s]

0568553
Contrato Número: 0568553
0568753
Contrato Número: 0568753
0569107
Contrato Número: 0569107
0569271
Contrato Número: 0569271
0569332
Contrato Número: 0569332
0569339
Contrato Número: 0569339
0569340
Contrato Número: 0569340
0569494
Contrato Número: 0569494
0569646
Contrato Número: 0569646
0569809
Contrato Número: 0569809
0569948
Contrato Número: 0569948
4027596
Contrato Número: 4027596
4111255
Contrato Número: 4111255
4131177
Contrato Número: 4131177
4236166
Contrato Número: 4236166
4294281
Contrato Número: 4294281
4351086
Contrato Número: 4351086


Processando grupos:  17%|█▋        | 1/6 [00:00<00:02,  2.38it/s]

4402558
Contrato Número: 4402558
4422407
Contrato Número: 4422407
4442873
Contrato Número: 4442873
4505066
Contrato Número: 4505066
4536806
Contrato Número: 4536806
4545492
Contrato Número: 4545492
4566363
Contrato Número: 4566363
4566399
Contrato Número: 4566399
4566538
Contrato Número: 4566538
4566549
Contrato Número: 4566549
4566550
Contrato Número: 4566550
4567217
Contrato Número: 4567217
4567354
Contrato Número: 4567354
4567375
Contrato Número: 4567375
4567516
Contrato Número: 4567516
4567597
Contrato Número: 4567597
4567607
Contrato Número: 4567607
4567815
Contrato Número: 4567815


Processando grupos:  33%|███▎      | 2/6 [00:01<00:02,  1.70it/s]

...

4572464
Contrato Número: 4572464
4572466
Contrato Número: 4572466
4573615
Contrato Número: 4573615
4574390
Contrato Número: 4574390
4574935
Contrato Número: 4574935
4576131
Contrato Número: 4576131
4578059
Contrato Número: 4578059
4578141
Contrato Número: 4578141
4593130
Contrato Número: 4593130
4634050
Contrato Número: 4634050
4641404
Contrato Número: 4641404
4661130
Contrato Número: 4661130
4671539
Contrato Número: 4671539
4674936
Contrato Número: 4674936


Processando grupos:  83%|████████▎ | 5/6 [00:03<00:00,  1.67it/s]

4691635
Contrato Número: 4691635
4695103
Contrato Número: 4695103
4933466
Contrato Número: 4933466
4933467
Contrato Número: 4933467
4936356
Contrato Número: 4936356
4959097
Contrato Número: 4959097
4965924
Contrato Número: 4965924
4965926
Contrato Número: 4965926
5001428
Contrato Número: 5001428
5019666
Contrato Número: 5019666
5029044
Contrato Número: 5029044
5029439
Contrato Número: 5029439
5030472
Contrato Número: 5030472
5030513
Contrato Número: 5030513
5036178
Contrato Número: 5036178
5047263
Contrato Número: 5047263


Processando grupos: 100%|██████████| 6/6 [00:03<00:00,  1.80it/s]


1